# Configuring Zarr to Vitessce Widget
Script that converts a pre-processed AnnData-based Zarr object into a Vitessce widget using Python. 

Proofchecking that our Zarr file is valid before deploying it to Github Pages. 

In [1]:
import os
from os.path import join, isfile, isdir
from urllib.request import urlretrieve
from anndata import read_h5ad
import scanpy as sc

from vitessce import (
    VitessceConfig,
    Component as cm,
    CoordinationType as ct,
    AnnDataWrapper,
)
from vitessce.data_utils import (
    optimize_adata,
    VAR_CHUNK_SIZE,
)

In [2]:
# change the directory to general to access data 
path = "/Users/iriskwon/Desktop/mouse-cereb-dev/mouse-cereb-dev"
os.chdir(path)

# verify the change
print("New working directory:", os.getcwd())

New working directory: /Users/iriskwon/Desktop/mouse-cereb-dev/mouse-cereb-dev


In [3]:
zarr_filepath = join("public/data", "E11_Scaled.zarr")

In [4]:
filepath_zarr_e11_1 = join("public/data", "E11_1.zarr")

In [5]:
del(vc)

NameError: name 'vc' is not defined

# Single Slice: 

In [6]:
from vitessce import VitessceConfig, ViewType as vt, CoordinationType as ct, AnnDataWrapper

# Set up the config
vc = VitessceConfig(schema_version="1.0.15", name="E11 STARmap slice 1")

# Define Zarr path
zarr_path = "public/data/E11_1.zarr"

# Create wrapper for spatial and expression data
w = AnnDataWrapper(
    adata_path=zarr_path,
    obs_embedding_paths=["obsm/X_umap"],
    obs_embedding_names=["UMAP"],
    obs_feature_matrix_path="X",
    obs_locations_path="obsm/spatial",
    obs_set_paths=["obs/cell_types", "obs/finer_cell_types", "obs/leiden"],
    obs_set_names=["Cell Type", "Finer Cell Types", "Leiden Clusters"]
)

# Add dataset and wrapper
dataset = vc.add_dataset(name="E11_Slice1").add_object(w)

# Coordination for embedding
embedding_scope = vc.add_coordination_by_dict({
    ct.EMBEDDING_TYPE: "UMAP"
})

# === Add views ===
desc = vc.add_view(vt.DESCRIPTION, dataset=dataset).set_props(description="My E11 STARmap data.")
spatial = vc.add_view(vt.SPATIAL, dataset=dataset)
scatter = vc.add_view(vt.SCATTERPLOT, dataset=dataset).use_coordination_by_dict(embedding_scope)
genes = vc.add_view(vt.FEATURE_LIST, dataset=dataset)
cell_sets = vc.add_view(vt.OBS_SETS, dataset=dataset)
heatmap = vc.add_view(vt.HEATMAP, dataset=dataset)

# Add spatial layer styling and spatial coordination AFTER spatial is defined
spatial_segmentation_layer_value = {
    "opacity": 1,
    "radius": 2,
    "visible": True,
    "stroked": False
}

vc.link_views(
    [spatial],
    [ct.SPATIAL_ZOOM, ct.SPATIAL_TARGET_X, ct.SPATIAL_TARGET_Y, ct.SPATIAL_SEGMENTATION_LAYER],
    [1.5, 500, 130, spatial_segmentation_layer_value]
)

# Layout
vc.layout((desc | cell_sets | spatial) / (genes | scatter | heatmap))
vw = vc.widget()
vw


VitessceWidget(config={'version': '1.0.15', 'name': 'E11 STARmap slice 1', 'description': '', 'datasets': [{'u…

# Multiple Slices:

In [15]:
from vitessce import VitessceConfig, ViewType as vt, CoordinationType as ct, AnnDataWrapper

# Set up the config
vc = VitessceConfig(schema_version="1.0.15", name="E11 STARmap")

# Define Zarr path
zarr_path = "public/data/E11_Normalized.zarr"

# Create wrapper for spatial and expression data
w = AnnDataWrapper(
    adata_path=zarr_path,
    obs_embedding_paths=["obsm/X_umap"],
    obs_embedding_names=["UMAP"],
    obs_feature_matrix_path="X",
    obs_locations_path="obsm/spatial",
    obs_set_paths=["obs/cell_types", "obs/library_id", "obs/leiden"],
    obs_set_names=["Cell Type", "Coronal Slice A-P", "Leiden Clusters"]
)

# Add dataset and wrapper
dataset = vc.add_dataset(name="E11").add_object(w)

# Coordination for embedding
embedding_scope = vc.add_coordination_by_dict({
    ct.EMBEDDING_TYPE: "UMAP"
})

# === Add views ===
desc = vc.add_view(vt.DESCRIPTION, dataset=dataset).set_props(description="My E11 STARmap data.")
spatial = vc.add_view(vt.SPATIAL, dataset=dataset)
scatter = vc.add_view(vt.SCATTERPLOT, dataset=dataset).use_coordination_by_dict(embedding_scope)
genes = vc.add_view(vt.FEATURE_LIST, dataset=dataset)
cell_sets = vc.add_view(vt.OBS_SETS, dataset=dataset)
heatmap = vc.add_view(vt.HEATMAP, dataset=dataset)

# Add spatial layer styling and spatial coordination AFTER spatial is defined
spatial_segmentation_layer_value = {
    "opacity": 1,
    "radius": 2,
    "visible": True,
    "stroked": False
}

vc.link_views(
    [spatial],
    [ct.SPATIAL_ZOOM, ct.SPATIAL_TARGET_X, ct.SPATIAL_TARGET_Y, ct.SPATIAL_SEGMENTATION_LAYER],
    [1.5, 259, 500, spatial_segmentation_layer_value]
)

# Layout
vc.layout((desc / cell_sets / genes) | spatial | (scatter | heatmap))
vw = vc.widget()
vw


VitessceWidget(config={'version': '1.0.15', 'name': 'E11 STARmap', 'description': '', 'datasets': [{'uid': 'A'…

# Save Vitessce Config to JSON: 

In [12]:
# Export your config to a Python dictionary
config_dict = vc.to_dict(base_url="public/data/")

# Save it as a JSON file
import json
with open("config.json", "w") as f:
    json.dump(config_dict, f, indent=2)

# Write it to a JS file for your React app
with open("src/my-view-config.js", "w") as f:
    f.write("export const myViewConfig = ")
    json.dump(config_dict, f, indent=2)
    f.write(";")
